<a href="https://colab.research.google.com/github/elizabethavargas/Dataset-Description-Generation/blob/main/create_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Create Dataset

In [1]:
# packages
import random
import requests
import pandas as pd
from io import BytesIO

In [ ]:
https://data.cityofnewyork.us/api/v3/views/8wbx-tsch/query.json

https://data.cityofnewyork.us/api/v3/views/8wbx-tsch/query.json?$$app_token=L76aBvmvvFwme9Q46GQJ3qtf8

#PaintThinner3!

### Get list of datasets on NYC Open Data

In [2]:
# Base URL for the NYC Open Data Socrata API
base_url = "https://data.cityofnewyork.us/api/views.json"

try:
    response = requests.get(base_url)
    response.raise_for_status()  # Raise an exception for bad status codes
    datasets_data = response.json()

    # Extract id and name for each dataset
    datasets_list = []
    for dataset in datasets_data:
        if 'id' in dataset and 'name' in dataset:
            datasets_list.append({'id': dataset['id'], 'name': dataset['name']})

    # Print confirmation message
    print(f"Successfully listed {len(datasets_list)} datasets.")

except requests.exceptions.RequestException as e:
    print(f"Error fetching data: {e}")
except ValueError:
    print("Error decoding JSON response. The response might not be in JSON format.")

Successfully listed 3008 datasets.


In [3]:
# Randomly select 10 datasets
num_datasets_to_query = 10
selected_datasets = random.sample(datasets_list, num_datasets_to_query)
selected_datasets

[{'id': 'gfqj-f768', 'name': '2018 Central Park Squirrel Census - Stories'},
 {'id': '84ax-hg3y',
  'name': 'Full-Time and FTE Headcount including Covered Organizations'},
 {'id': 'fn8u-htpz',
  'name': '2020-2021 EMS Transport Full Year - Race, SWD, YOB'},
 {'id': '6anw-twe4', 'name': 'Local Law 44 - Development Team'},
 {'id': 'qf2u-z2br',
  'name': '2016-2017 Diversity Report - Pre-Kindergarten, K-8 & Grades 9-12 District, Schools, Special Programs, Diversity Efforts, Admissions Methods'},
 {'id': 'wvxf-dwi5', 'name': 'Housing Maintenance Code Violations'},
 {'id': '3kfa-rvez', 'name': 'NYC Pool Inspections'},
 {'id': 'vxxs-iyt2',
  'name': '2015-16 Guidance Counselor Reporting - Social Worker Data'},
 {'id': 'xni9-ncns',
  'name': '2018-2019 Diversity Report - Pre-Kindergarten, K-8 & Grades 9-12 District, Schools, Special Programs, Diversity Efforts, Admissions Methods'},
 {'id': 'ywiv-5gyw', 'name': '2012 D75 School Survey Data'}]

In [4]:
from difflib import SequenceMatcher

def find_closest_sheet_name(sheet_names, target_names):
    """
    Finds the sheet name in a list that is most similar to one of the target names.

    Args:
        sheet_names (list): A list of sheet names (strings).
        target_names (list): A list of target names (strings) to compare against.

    Returns:
        str: The sheet name from sheet_names that is most similar to a target_name,
             or None if no sheet names are provided.
    """
    if not sheet_names:
        return None

    best_match = None
    highest_similarity = -1

    for sheet_name in sheet_names:
        for target_name in target_names:
            similarity = SequenceMatcher(None, sheet_name.lower(), target_name.lower()).ratio()
            if similarity > highest_similarity:
                highest_similarity = similarity
                best_match = sheet_name

    return best_match



In [5]:
import pandas as pd
import json

def format_example_rows(df, max_chars=150):
    """
    Formats the first n rows of a DataFrame as clean JSON-like examples for LLM prompts.
    - Truncates long strings/numbers.
    - Removes NaN values.
    """
    example = {}
    for col, val in df.iloc[0].items():
        # Simplify nested dicts or long text
        if isinstance(val, (dict, list)):
            val_str = json.dumps(val)
        else:
            val_str = str(val)
        if len(val_str) > max_chars:
            val_str = val_str[:max_chars] + "..."
        if val_str not in ("nan", "None", ""):
            example[col] = val_str
    formatted = f"Example row: {json.dumps(example, ensure_ascii=False, indent=2)}\n"
    return formatted

In [18]:
datasets = []
for dataset in selected_datasets:
    dataset_id = dataset['id']
    dataset_name = dataset['name']
    print(f"Querying dataset: {dataset_name} (ID: {dataset_id}) ---")

    # Construct the API endpoint URL for the dataset and metadata
    dataset_url = f"https://data.cityofnewyork.us/resource/{dataset_id}.json" #f"https://data.cityofnewyork.us/api/v3/views/{dataset_id}/query.json?$$app_token=L76aBvmvvFwme9Q46GQJ3qtf8" #f"https://data.cityofnewyork.us/resource/{dataset_id}.json"
    metadata_url = f"https://data.cityofnewyork.us/api/views/{dataset_id}.json"


    try:
        print("Fetching data...")
        data_response = requests.get(dataset_url, headers={"X-App-Token": "L76aBvmvvFwme9Q46GQJ3qtf8"}, params={'$limit': 2})

        data_response.raise_for_status()  # Raise an exception for bad status codes
        data = data_response.json()

        if data:
            df = pd.DataFrame(data)
            print(f"Successfully retrieved data for {dataset_name}:")
            display(df.head())
        else:
            print(f"No data returned for dataset: {dataset_name}")


    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for {dataset_name}: {e}")
    except ValueError:
        print(f"Error decoding JSON response for data of {dataset_name}. The response might not be in JSON format.")
    except Exception as e:
        print(f"An unexpected error occurred while processing data for {dataset_name}: {e}")

    try:
        print("Fetching metadata...")
        metadata_response = requests.get(metadata_url)
        metadata_response.raise_for_status() # Raise an exception for bad status codes
        metadata = metadata_response.json()

        if metadata:
            # Get category, description, agency, and tags
            category = metadata.get('category', 'N/A')
            description = metadata.get('description', 'N/A')
            agency = metadata.get('metadata', {}).get('custom_fields', {}).get('Dataset Information', {}).get('Agency', 'N/A')
            tags = metadata.get('tags', [])
            title = metadata.get('name', 'N/A') # Assuming 'name' is the title

            prompt = f"Write an improved dataset description for the dataset titled {title} with the category {category}, and by the agency {agency}."

            if tags:
              prompt += f" The tags are {tags}."
            if description:
              cleaned_description = "\n".join([line for line in description.splitlines() if line.strip()])
              prompt += f""" The current description is "{cleaned_description}"."""

            prompt += format_example_rows(df)

            # add column descriptions
            column_info = {}
            for column in metadata.get('columns'):
              column_name = column.get('name', 'N/A')
              column_description = column.get('description', '')
              if column_description != '':
                prompt += f"{column_name}: {column_description}\n"
                column_info[column_name] = column_description

            # --- Data Dictionary ---
            attachments = metadata.get('metadata', {}).get('attachments', [])
            if attachments and not column_info:
                for attach in attachments:
                    file_id = attach.get('assetId')
                    filename = attach.get('filename', 'data_dictionary.xlsx')
                    if file_id:
                        file_url = f"https://data.cityofnewyork.us/api/views/{dataset_id}/files/{file_id}?download=true&filename={filename}"
                        print(f"Downloading data dictionary: {filename} ...")
                        try:
                            file_response = requests.get(file_url)
                            file_response.raise_for_status()
                            excel_file = BytesIO(file_response.content)
                            xls = pd.ExcelFile(excel_file)
                            print(f"Sheets in data dictionary: {xls.sheet_names}")
                            target_names = ["data dictionary", "column descriptions", "column definitions"]
                            closest_sheet = find_closest_sheet_name(xls.sheet_names, target_names)
                            print("First few rows of data dictionary:")
                            try:
                                # Try reading the first sheet
                                df_preview = pd.read_excel(xls, sheet_name=closest_sheet, header=None)

                                # --- detect header row dynamically ---
                                def detect_header_row(df_full, max_rows=10):
                                    """Return the index of the most likely header row (0-based)."""
                                    for i in range(min(max_rows, len(df_full))):
                                        row = df_full.iloc[i].astype(str).str.lower()
                                        keywords = ["column", "field", "name", "description", "definition"]
                                        matches = sum(any(k in cell for k in keywords) for cell in row)
                                        if matches >= 2:  # at least 2 useful terms in the row
                                            return i
                                    return 0  # fallback

                                # --- detect and reload with the correct header row ---
                                header_idx = detect_header_row(df_preview)
                                if header_idx > 0:
                                    print(f"Detected header likely at row {header_idx} in sheet '{closest_sheet}'. Re-reading sheet.")
                                df_dict = pd.read_excel(xls, sheet_name=closest_sheet, header=header_idx)
                                display(df_dict.head())

                                 # find column name and description rows
                                name_column = find_closest_sheet_name(list(df_dict.columns), ['column name', 'column', 'field', 'field name'])
                                description_column = find_closest_sheet_name(list(df_dict.columns), ['description', 'definiton', 'column definition', 'field description'])
                                if name_column and description_column and name_column != description_column:
                                  for idx, row in df_dict.iterrows():
                                    if row[description_column] != '':
                                      prompt += f"{row[name_column]}: {row[description_column]}\n"
                                      column_info[row[name_column]] = row[description_column]
                                print(column_info)


                            except Exception as e:
                                print(f"Error reading sheet '{closest_sheet}': {e}")

                        except Exception as e:
                            print(f"Error opening data dictionary: {e}")
                    else:
                        print(f"No file ID found for {filename}")
            else:
                print("No data dictionary attachment found.")

            print(f"Generated prompt:\n{prompt}")

            dataset_info = {
                'dataset_id': dataset_id,
                'dataset_name': dataset_name,
                'category': category,
                'description': cleaned_description,
                'agency': agency,
                'tags': tags,
                'column_info': column_info,
                'prompt_1': prompt
            }
            datasets.append(dataset_info)
        else:
            break
            print(f"No metadata returned for dataset: {dataset_name}")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching metadata for {dataset_name}: {e}")
    except ValueError:
        print(f"Error decoding JSON response for metadata of {dataset_name}. The response might not be in JSON format.")
    except Exception as e:
        print(f"An unexpected error occurred while processing metadata for {dataset_name}: {e}")

Querying dataset: 2018 Central Park Squirrel Census - Stories (ID: gfqj-f768) ---
Fetching data...
Successfully retrieved data for 2018 Central Park Squirrel Census - Stories:


,hectare,shift,date,note_squirrel_park_stories,story_topic_squirrel,story_topic_park_experience
0,01A,PM,10142018,Observed a squirrel with a cache of peanuts th...,True,NaN
1,01A,AM,10182018,"Lots of pedestrians and vehicular traffic, foo...",NaN,True


Fetching metadata...
No data dictionary attachment found.
Generated prompt:
Write an improved dataset description for the dataset titled 2018 Central Park Squirrel Census - Stories with the category Environment, and by the agency Other. The tags are ['squirrel census']. The current description is "The Squirrel Census (https://www.thesquirrelcensus.com/) is a multimedia science, design, and storytelling project focusing on the Eastern gray (Sciurus carolinensis). They count squirrels and present their findings to the public. 
This table contains stories, observations, and notes written on tally sheets by Squirrel Sighters during the Census. Individual stories have been tagged with common story themes, such as “squirrel experience,” “park experience,” “dogs,” “other animals,” “accidental poems,” and “squirrels acting odd.”".Example row: {
  "hectare": "01A",
  "shift": "PM",
  "date": "10142018",
  "note_squirrel_park_stories": "Observed a squirrel with a cache of peanuts that he was eat

,pub_date,agency,agency_name,fyear,unipedname,agy_grp,city,total
0,20200416,002,Mayoralty,2021,Civilian,Mayoral Agencies & Elected Officials,1106,1331
1,20200416,002,Mayoralty,2022,Civilian,Mayoral Agencies & Elected Officials,1100,1325


Fetching metadata...
No data dictionary attachment found.
Generated prompt:
Write an improved dataset description for the dataset titled Full-Time and FTE Headcount including Covered Organizations with the category City Government, and by the agency Mayor's Office of Management & Budget (OMB). The tags are ['nyc omb', 'mayor’s message', 'mm', 'exhibit 5', 'full-time and full-time equivalent (fte) headcount', 'mayoral agencies and elected agencies', 'covered organizations', 'financial plan detail', 'fpd']. The current description is "This dataset contains estimated fiscal year-end headcount information for full-time and full-time equivalent employees (FTE) for Mayoral Agencies and Covered Organizations (updated twice a year). The information is summarized by agency, fiscal year, personnel type and funding.".Example row: {
  "pub_date": "20200416",
  "agency": "002",
  "agency_name": "Mayoralty",
  "fyear": "2021",
  "unipedname": "Civilian",
  "agy_grp": "Mayoral Agencies & Elected Offi

...

Fetching metadata...
Sheets in data dictionary: ['Summary', 'Parent %', 'Parent Scores', 'Teacher %', 'Parent # of Responses', 'Teacher Scores', 'Teacher # of Responses', 'Student %', 'Student Scores', 'Student # of Responses', 'Dataset Info', 'Column Info', 'Dataset Revision History', 'Data Dictionary']
First few rows of data dictionary:
Detected header likely at row 3 in sheet 'Data Dictionary'. Re-reading sheet.


,Field Name,Field Description
0,dbn,School identification code (district borough n...
1,locationname,School name
2,rr_s,Student Response Rate
3,rr_t,Teacher Response Rate
4,rr_p,Parent Response Rate


{'dbn': 'School identification code (district borough number)', 'locationname': 'School name', 'rr_s': 'Student Response Rate', 'rr_t': 'Teacher Response Rate', 'rr_p': 'Parent Response Rate', 'N_s': 'Number of student respondents', 'N_t': 'Number of teacher respondents', 'N_p': 'Number of parent respondents', 'nr_s': 'Number of eligible students', 'nr_t': 'Number of eligible teachers', 'nr_p': 'Number of eligible parents (need to add to template)', 'saf_p_12': 'Safety and Respect score based on parent responses', 'com_p_12': 'Communication score based on parent responses', 'eng_p_12': 'Engagement score based on parent responses', 'aca_p_12': 'Academic expectations score based on parent responses', 'saf_t_12': 'Safety and Respect score based on teacher responses', 'com_t_12': 'Communication score based on teacher responses', 'eng_t_12': 'Engagement score based on teacher responses', 'aca_t_12': 'Academic expectations score based on teacher responses', 'saf_s_12': 'Safety and Respect sc

,Field Name,Field Description
0,dbn,School identification code (district borough n...
1,locationname,School name
2,rr_s,Student Response Rate
3,rr_t,Teacher Response Rate
4,rr_p,Parent Response Rate


{'dbn': 'School identification code (district borough number)', 'locationname': 'School name', 'rr_s': 'Student Response Rate', 'rr_t': 'Teacher Response Rate', 'rr_p': 'Parent Response Rate', 'N_s': 'Number of student respondents', 'N_t': 'Number of teacher respondents', 'N_p': 'Number of parent respondents', 'nr_s': 'Number of eligible students', 'nr_t': 'Number of eligible teachers', 'nr_p': 'Number of eligible parents (need to add to template)', 'saf_p_12': 'Safety and Respect score based on parent responses', 'com_p_12': 'Communication score based on parent responses', 'eng_p_12': 'Engagement score based on parent responses', 'aca_p_12': 'Academic expectations score based on parent responses', 'saf_t_12': 'Safety and Respect score based on teacher responses', 'com_t_12': 'Communication score based on teacher responses', 'eng_t_12': 'Engagement score based on teacher responses', 'aca_t_12': 'Academic expectations score based on teacher responses', 'saf_s_12': 'Safety and Respect sc

In [11]:
df_dict.columns

Index(['Field Name', 'Field Description'], dtype='object')

In [16]:
 # find column name and description rows
name_column = find_closest_sheet_name(list(df_dict.columns), ['column name', 'column', 'field', 'field name'])
description_column = find_closest_sheet_name(list(df_dict.columns), ['description', 'definiton', 'column definition', 'field description'])
if name_column and description_column and name_column != description_column:
  for idx, row in df_dict.iterrows():
    if row[description_column] != '':
      prompt += f"{row[name_column]}: {row[description_column]}\n"
      column_info[row[name_column]] = row[description_column]

print(column_info)



{'dbn': 'School identification code (district borough number)', 'locationname': 'School name', 'rr_s': 'Student Response Rate', 'rr_t': 'Teacher Response Rate', 'rr_p': 'Parent Response Rate', 'N_s': 'Number of student respondents', 'N_t': 'Number of teacher respondents', 'N_p': 'Number of parent respondents', 'nr_s': 'Number of eligible students', 'nr_t': 'Number of eligible teachers', 'nr_p': 'Number of eligible parents (need to add to template)', 'saf_p_12': 'Safety and Respect score based on parent responses', 'com_p_12': 'Communication score based on parent responses', 'eng_p_12': 'Engagement score based on parent responses', 'aca_p_12': 'Academic expectations score based on parent responses', 'saf_t_12': 'Safety and Respect score based on teacher responses', 'com_t_12': 'Communication score based on teacher responses', 'eng_t_12': 'Engagement score based on teacher responses', 'aca_t_12': 'Academic expectations score based on teacher responses', 'saf_s_12': 'Safety and Respect sc

In [19]:
datasets

[{'dataset_id': 'gfqj-f768',
  'dataset_name': '2018 Central Park Squirrel Census - Stories',
  'category': 'Environment',
  'description': 'The Squirrel Census (https://www.thesquirrelcensus.com/) is a multimedia science, design, and storytelling project focusing on the Eastern gray (Sciurus carolinensis). They count squirrels and present their findings to the public. \nThis table contains stories, observations, and notes written on tally sheets by Squirrel Sighters during the Census. Individual stories have been tagged with common story themes, such as “squirrel experience,” “park experience,” “dogs,” “other animals,” “accidental poems,” and “squirrels acting odd.”',
  'agency': 'Other',
  'tags': ['squirrel census'],
  'column_info': {'Hectare': 'ID tag, which is derived from the hectare grid used to divide and count the park area. One axis that runs predominantly north-to-south is numerical (1-42), and the axis that runs predominantly east-to-west ',
   'Shift': 'Value is either "A